In [2]:
import pandas as pd
import sqlite3
filename = 'chinook.db'
connection= sqlite3.connect(filename)

In [3]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",con=connection)

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


In [4]:
query = "SELECT COUNT(*) FROM INVOICES"
pd.read_sql(query,con=connection)

,COUNT(*)
0,412


### Check the number of Customers from each Country

In [12]:
# Q1 using sqlite3
query = "SELECT Count(CustomerId), Country FROM CUSTOMERS GROUP BY Country order by COUNT(CustomerId) DESC"
cursor = connection.cursor()
value=cursor.execute(query)
value.fetchall()

[(13, 'USA'),
 (8, 'Canada'),
 (5, 'France'),
 (5, 'Brazil'),
 (4, 'Germany'),
 (3, 'United Kingdom'),
 (2, 'Portugal'),
 (2, 'India'),
 (2, 'Czech Republic'),
 (1, 'Sweden'),
 (1, 'Spain'),
 (1, 'Poland'),
 (1, 'Norway'),
 (1, 'Netherlands'),
 (1, 'Italy'),
 (1, 'Ireland'),
 (1, 'Hungary'),
 (1, 'Finland'),
 (1, 'Denmark'),
 (1, 'Chile'),
 (1, 'Belgium'),
 (1, 'Austria'),
 (1, 'Australia'),
 (1, 'Argentina')]

In [4]:
invoices_tbl=pd.read_sql("SELECT * FROM CUSTOMERS",con=connection)
country = invoices_tbl.groupby('Country')
country_count = country.count()
country_count

,CustomerId,FirstName,LastName,Company,Address,City,State,PostalCode,Phone,Fax,Email,SupportRepId
Country,,,,,,,,,,,,
Argentina,1,1,1,0,1,1,0,1,1,0,1,1
Australia,1,1,1,0,1,1,1,1,1,0,1,1
Austria,1,1,1,0,1,1,0,1,1,0,1,1
Belgium,1,1,1,0,1,1,0,1,1,0,1,1
Brazil,5,5,5,4,5,5,5,5,5,5,5,5
Canada,8,8,8,2,8,8,8,8,8,2,8,8
Chile,1,1,1,0,1,1,0,0,1,0,1,1
Czech Republic,2,2,2,1,2,2,0,2,2,1,2,2
Denmark,1,1,1,0,1,1,0,1,1,0,1,1


### Finding out the maximum duration of a track.

In [17]:
query = "SELECT Milliseconds, Name, TrackId FROM TRACKS order by Milliseconds DESC Limit 1"
cursor = connection.cursor()
value=cursor.execute(query)
value.fetchall()

[(5286953, 'Occupation / Precipice', 2820)]

In [50]:
track_table = pd.read_sql("SELECT * FROM TRACKS",con=connection)
sorted_table = track_table.sort_values('Milliseconds', ascending = False)
maximum_title = sorted_table.iloc[0]
print('The longest running track: \n',maximum_title)

The longest running track: 
 TrackId                           2820
Name            Occupation / Precipice
AlbumId                            227
MediaTypeId                          3
GenreId                             19
Composer                          None
Milliseconds                   5286953
Bytes                       1054423946
UnitPrice                         1.99
Name: 2819, dtype: object


### Finding out the Country with the Highest Sales

In [19]:
query = "SELECT Count(InvoiceId), BillingCountry FROM INVOICES GROUP BY BillingCountry order by COUNT(InvoiceId) DESC limit 1"
cursor = connection.cursor()
value=cursor.execute(query)
value.fetchall()

[(91, 'USA')]

In [35]:
invoices_tbl=pd.read_sql("SELECT * FROM INVOICES",con=connection)
country = invoices_tbl.groupby('BillingCountry')
country_count = country.count()
highest = country_count.sort_values('Total', ascending = False)
h_country = highest.iloc[0]
print('The country with the highest number of sales: \n',h_country)

The country with the highest number of sales: 
 InvoiceId            91
CustomerId           91
InvoiceDate          91
BillingAddress       91
BillingCity          91
BillingState         91
BillingPostalCode    91
Total                91
Name: USA, dtype: int64


### Find out the percentage of sales per music genre

In [3]:
query = 'select GenreId,(sum(Total)/20848.62*100) from INVOICES join Invoice_items on Invoice_items.InvoiceId = Invoices.InvoiceId join Tracks on Tracks.trackId = Invoice_items.trackId  Group By GenreId order by (sum(Total)/20848.62*100) desc'
cursor = connection.cursor()
value=cursor.execute(query)
pd.DataFrame(value.fetchall())

,0,1
0,1,37.028926
1,7,16.656018
2,3,10.039657
3,4,9.409064
4,19,3.922130
5,2,3.580381
6,21,2.612211
7,6,2.060856
8,14,1.624184
9,8,1.595501


In [6]:
invoice_tbl = pd.read_sql("Select * from INVOICES", con=connection)
invoice_items_tbl = pd.read_sql("Select * from Invoice_items", con = connection)

invoice_merge = pd.merge(invoice_tbl, invoice_items_tbl)

invoice_merge_df = pd.DataFrame(invoice_merge)

tracks_tbl = pd.read_sql("select * from TRACKS", con = connection)

tracks_merge = pd.merge(tracks_tbl, invoice_merge_df)

tracks_merge_df = pd.DataFrame(tracks_merge)

groupby_invoices =tracks_merge_df.groupby('GenreId')

total = tracks_merge_df['Total'].sum()
print("Percentage:\n", groupby_invoices['Total'].sum().sort_values(ascending= False)/total*100)

Percentage:
 GenreId
1     37.028926
7     16.656018
3     10.039657
4      9.409064
19     3.922130
2      3.580381
21     2.612211
6      2.060856
14     1.624184
8      1.595501
24     1.520676
10     1.163386
9      1.149956
23     1.012873
20     0.953876
16     0.873823
17     0.798182
13     0.774008
15     0.717649
12     0.664792
22     0.538645
18     0.491208
11     0.413121
5      0.398875
Name: Total, dtype: float64


### Find out the spendings of Customer 12

In [37]:
query = "SELECT Customers.FirstName, SUM(INVOICES.Total) FROM CUSTOMERS INNER JOIN INVOICES ON CUSTOMERS.CustomerId = 12 AND INVOICES.CustomerId = 12"
cursor = connection.cursor()
value=cursor.execute(query)
value.fetchall()

[('Roberto', 37.62)]

In [8]:
invoices_tbl=pd.read_sql("SELECT * FROM INVOICES WHERE CustomerID == 12",con=connection)
customer_tbl=pd.read_sql("SELECT * FROM CUSTOMERS WHERE CustomerID == 12",con=connection)
combine_tbl = pd.merge(invoices_tbl,customer_tbl)
spending=combine_tbl.loc['Spending'] = pd.Series(combine_tbl['Total'].sum(), index = ['Total'])
print(spending)

Total    37.62
dtype: float64


### Find out the artist the with most rock genre songs

In [47]:
query = "SELECT TRACKS.Composer, COUNT(TRACKS.Name) FROM tracks INNER JOIN GENRES ON GENRES.GenreId = 1 AND TRACKS.GenreId = 1 GROUP BY TRACKS.Composer order by COUNT(TRACKS.Name) DESC limit 2"
cursor = connection.cursor()
value=cursor.execute(query)
value.fetchall()

[(None, 168), ('U2', 44)]

In [62]:
tracks_tbl=pd.read_sql("SELECT * FROM TRACKS WHERE GenreId == 1",con=connection)
albums_tbl=pd.read_sql("SELECT * FROM ALBUMS",con=connection)
tbl_rock = pd.merge(tracks_tbl,albums_tbl)
artist = tbl_rock.groupby('ArtistId')
artist.count = artist.count()
artist.sorted = artist.count.sort_values('TrackId', ascending = False)
artist_with_most_songs=pd.read_sql("SELECT * FROM ARTISTS WHERE ArtistId = 22",con=connection)
print('The Artist with most rock songs was', artist_with_most_songs['Name'])

The Artist with most rock songs was 0    Led Zeppelin
Name: Name, dtype: object
